In [1]:
import os
from google.cloud.sql.connector import Connector
from sqlalchemy import create_engine, text
import pg8000.native
import pandas as pd

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users/tinma/OneDrive/Escritorio/application_default_credentials.json'

connector = Connector()

def getconn() -> pg8000.native.Connection:
    conn: pg8000.native.Connection = connector.connect(
        "semiotic-pact-388619:us-central1:tradingbot",
        "pg8000",
        user="postgres",
        password="GuruSat.3",
        db="criptos"
    )
    return conn

connection = create_engine('postgresql+pg8000://',creator=getconn)

In [7]:
path = 'C:/Users/tinma/OneDrive/Escritorio/tradingbot/csvs/historical/byday/6years/'
pairs = os.listdir(path)
pairs = pd.DataFrame(pairs,columns=['pairs'])

downs = pairs[pairs['pairs'].str.contains('DOWN')]
ups = pairs[pairs['pairs'].str.contains('UP')]
ups_downs = pd.concat([ups, downs], ignore_index=True)
ups_downs = ups_downs[ups_downs['pairs'].str.contains('SUPERUSDT')==False]

for x in ups_downs['pairs']:
    pairs = pairs[pairs['pairs'].str.contains(x)==False]

for pair in pairs['pairs']:
    with connection.connect() as conn:
        conn.execute(text(""" DELETE FROM "{pair}" WHERE "Tiempo" IN (SELECT "Tiempo" FROM (SELECT "Tiempo", ROW_NUMBER() OVER( PARTITION BY "Tiempo" ORDER BY  "Tiempo" ) AS row_num FROM "{pair}" ) t WHERE t.row_num > 1 ) """.format(pair=pair)))